# Broadlink Device Testing

This notebook provides functionality to test Broadlink device commands for the kitchen hood.

## Device Discovery and Setup

This section discovers the Broadlink device and authenticates with it. The device information is loaded from the configuration file.

In [ ]:
import broadlink
import json
import base64

# Load device configuration
with open('config/devices/kitchen_hood.json', 'r') as f:
    config = json.load(f)

# Create device instance
device = broadlink.rm4pro(
    host=(config['broadlink']['host'], 80),
    mac=bytes.fromhex(config['broadlink']['mac'].replace(':', '')),
    devtype=int(config['broadlink']['device_class'], 16)
)

# Print device information
print(f"Device: {device}")
print(f"IP: {device.host}")
print(f"MAC: {device.mac.hex(':')}")
print(f"Type: {hex(device.devtype)}")
print(f"Device class: {device.__class__}")

# Authenticate with device
try:
    device.auth()
    print("Authentication successful")
except Exception as e:
    print(f"Authentication failed: {e}")

## Light On

This section tests the light control commands for the kitchen hood.

In [ ]:
def send_command(command_name, control_group=None):
    """Send a command to the device.
    
    Args:
        command_name: Name of the command to send
        control_group: Optional group name from config. If not provided, function will search all groups
    """
    # Find the command in the config structure
    action = None
    action_group = None
    
    if control_group and control_group in config['commands']:
        # Search only in the specified control group
        for act in config['commands'][control_group]['actions']:
            if act['name'] == command_name:
                action = act
                action_group = control_group
                break
    else:
        # Search in all control groups
        for group_name, group in config['commands'].items():
            for act in group['actions']:
                if act['name'] == command_name:
                    action = act
                    action_group = group_name
                    break
            if action:
                break
    
    if not action:
        print(f"Command '{command_name}' not found")
        return
    
    print(f"Sending command: {action['description']}")
    print(f"Control group: {action_group}")
    
    # Decode and send the RF code
    rf_code = base64.b64decode(action['rf_code'])
    print(f"Code={action['rf_code']}")
    print(f"Hex: {rf_code.hex()}")
    
    device.send_data(rf_code)
    print("Command sent successfully")

# Test light on command
send_command('light_off', 'light_control')


## Light Off

This section tests the light control commands for the kitchen hood.

In [ ]:
def send_command(command_name):
    """Send a command to the device."""
    command = config['commands'][command_name]
    print(f"Sending command: {command['description']}")
    
    # Decode and send the RF code
    rf_code = base64.b64decode(command['rf_code'])
    device.send_data(rf_code)
    print("Command sent successfully")

# Test light off command
send_command('light_off')

## Hood Up

This section tests the hood control commands for raising and lowering the hood.

In [ ]:
# Test hood up command
send_command('hood_speed1')

# send_command('hood_speed2')


## Hood Down

This section tests the hood control commands for raising and lowering the hood.

In [ ]:
# Test hood down command
send_command('hood_off')